In [2]:
import gzip
import csv
import math
import matplotlib.pyplot as plt
import numpy as np
import random
import sklearn
import string
from collections import defaultdict
from gensim.models import word2vec
from nltk.stem.porter import *
from sklearn import linear_model
from sklearn.manifold import TSNE

In [3]:
f = open("/Users/youzezheng/Desktop/Team-TBD/input/mbti_1.csv")

data = []
for l in csv.reader(f, delimiter=','):
    if l == ['type', 'posts']: continue
    data.append(l)

f.close()

In [4]:
len(data)

8675

In [5]:
wordCount = defaultdict(int)
punctuation = set(string.punctuation)
stemmer = PorterStemmer()
for type, post in data:
    r = ''.join(c for c in post.lower() if not c in punctuation)
    for w in r.split():
        w = stemmer.stem(w)
        wordCount[w] += 1
len(wordCount)

314610

In [6]:
counts = [(wordCount[w], w) for w in wordCount]
counts.sort()
counts.reverse()

In [7]:
words = [x[1] for x in counts[:2000]]

In [8]:
# Sentiment Analysis
wordId = dict(zip(words, range(len(words))))
wordSet = set(words)

In [9]:
def feature(datum):
    feat = [0]*len(words)
    r = ''.join(c for c in datum.lower() if not c in punctuation)
    for w in r.split():
        w = stemmer.stem(w)
        if w in words:
            feat[wordId[w]] += 1
    feat.append(1) # offset
    return feat

In [10]:
X = [feature(post) for _,post in data]

In [11]:
y = [type for type,post in data]

In [12]:
Ntrain = int(len(data)*0.8)
Nvalid = int(len(data)*0.2)

X_train = X[:Ntrain]
X_valid = X[Ntrain:]
y_train = y[:Ntrain]
y_valid = y[Ntrain:]

In [40]:
my_words = "My name is Hargen and I'm currently a second year student at University of California, San Diego. I am pursuing a double major in Mathematics - Computer Science under Department of Mathematics and Data Science under Halıcıoğlu Data Science Institute. I am interested in exploring more about Artificial Intelligence and Machine Learning. Currently, I am trying to build a more solid mathematical foundation for ML through coursework, so I can gain more maturity to understand ML concepts in depth. Meanwhile, I am building my technical skills to train ML/DL models by self-learning and hopefully my skillset could grow to a level where I can build meaningful ML projects and work in a ML lab. After a general introduction to ML during my visit at UC Berkeley, I am particularly drawn to problems related to Computer Vision and how we could apply Computer Vision techniques on robotics to make progress on Artificial General Intelligence. I think this is pretty cool! However, I am yet to explore more domain specific ML in my second year and hopefully I can determine my focus in the near future!"
my_words = feature(my_words)

In [ ]:
my_words = "Julie Andrews is warm, charismatic, and motivated. Her natural creativity and drive have likely contributed to her success as a performer. She is able to interpret each character in an original way and put in the work needed to do them each justice."
my_words = feature(my_words)

In [1]:
my_words = "After answering multiple questions for different situations I was able to obtain information about my personality. ENFJ is a very unique and has various skill sets that will allow me as individual to work in multiple roles. Occupations with good interpersonal skills that allow productive collaboration through any working process. Although this personality can be challenging it still has many benefits to help build careers successfully. There are many high rank positions that this particular personality type would be effective in. Some examples are sales, social services, community care, legal and the list goes on. In all of these roles there are a broad range of occupations that would fit perfectly based on the personality type. However, the ultimate focus is to reach their main dreams. Although they may see their selves as helpers or enablers, this creates the challenge. Throughout the course of years, I tend to find out more about myself. Every day is challenging and full of many surprises. However, the most challenging thing in my life is working with other personalities. Only to find myself considered an ENFJ with a temperament of NF. Yes, I’m open to others and would put myself last, but I now understand how to truly connect."
my_words = feature(my_words)

NameError: name 'feature' is not defined

In [14]:
from sklearn.linear_model import LogisticRegression

In [15]:
mod = LogisticRegression()

In [16]:
mod.fit(X_train, y_train)

/Users/youzezheng/anaconda3/envs/mbti/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [17]:
theta = mod.coef_[0]

In [18]:
weights = list(zip(theta, words + ['constant_feat']))

In [19]:
weights.sort()

In [20]:
weights[:10]

[(-0.12546811319840076, 'entp'),
 (-0.1098371010372366, 'intp'),
 (-0.09424702827567578, 'or'),
 (-0.08596554066227877, 'isfj'),
 (-0.08140379206357207, 'isfp'),
 (-0.07862031648822781, 'then'),
 (-0.070651050180706, 'an'),
 (-0.0677125399161527, 'cant'),
 (-0.0675110016193256, 'dont'),
 (-0.06509106275211089, 'onli')]

In [21]:
mod.predict([X[0]])

array(['INFJ'], dtype='<U4')

In [22]:
mod.score(X_valid, y_valid) # Validation accuracy

0.6172910662824207

### Do four binary classifications --> combine them together

In [38]:
my_words = "With such confidence, I reviewed the classical pieces that I practiced before and tried to see how composers used different patterns to set the emotion, and how the chords and progressions were arranged to create pleasant sounds. I even mimicked those patterns to write some variations of them, each expressing different emotions. Then I performed those variations on my cello and shared the recordings with my friends. Unlike mindless practicing, composing was quite fun for me and it brought me a strong sense of achievement. Moreover, creating small musical pieces became a hobby of mine where I could let my creativity and imagination fly."
my_words = feature(my_words)

In [34]:
my_words[:10]

[5, 8, 5, 4, 9, 5, 3, 3, 0, 2]

In [41]:
mod.predict([my_words])

array(['INTJ'], dtype='<U4')

In [ ]:
mod.score(X_train, y_train) # training accuracy

In [ ]:
predictions = [mod.predict([x]) for x in X_valid]
predictions[0][0]

In [ ]:
correct = 0
for pred,label in zip(predictions, y_valid):
    if pred[0] == label: correct +=1
correct / len(y_valid)